In [2]:
import keras as K
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.regularizers import l2, activity_l2,l1
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from scipy import misc
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import scipy

Using Theano backend.
Using gpu device 1: Tesla K20c (CNMeM is disabled, cuDNN 4007)


### Generate model with VGG16 feature extractors, set trainable false

In [3]:
temperature=2

def softmaxTemp(x):
    return K.softmax(x/temperature)

def getModel( output_dim):
    # output_dim: the number of classes (int)
    # return: compiled model (keras.engine.training.Model)
    
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-1].output
    
    out = Dense( output_dim, activation='softmax')( vgg_out )
    
    tl_model = Model( input=vgg_model.input, output=out)
    
    tl_model.layers[-2].activation=softmaxTemp
    
    for layer in tl_model.layers[0:-1]:
        layer.trainable = False            

    tl_model.compile(loss= "categorical_crossentropy", optimizer="adagrad", metrics=["acc"])
    tl_model.summary()
    return tl_model

### Loading all images into 'album'

In [4]:
def loadImages(path = '/mnt/cube/UT_/Urban_tribes/'):
    album = {}
    for item in listdir(path):
        category = item[0:4]
        if category == ".ipy":
            continue
        if category not in album:
            album[category] = []
        
        img = load_img(path+item)
        img = img_to_array(img)
        img = misc.imresize(img, (224, 224))
        img = scipy.misc.imrotate(img, 180)
        album[category].append(img)
    return album

In [5]:
album = loadImages()

In [6]:
# plt.imshow(album['goth'][0][:, :, 0])
# plt.show()

### Split 'album' into dataset
Album is split into training and testing input/outputs according to number of examples attribute

In [7]:
def make_dataset(album, numExamples):
    trn_inp = []
    trn_out = []
    tst_inp = []
    tst_out = []
    keys = album.keys()
    for key in keys:
        examples = album[key]
        l = len(examples)
        idx = np.random.choice(l, numExamples+30)
        for i in idx[:-5]:
            trn_inp.append(examples[i])
            trn_out.append(keys.index(key))
        for i in idx[-5:]:
            tst_inp.append(examples[i])
            tst_out.append(keys.index(key))
    return [trn_inp, trn_out, tst_inp, tst_out]      

In [8]:
# plt.imshow(album['bike'][0][:, :, 0])
# # plt.show()
# album.keys()

In [9]:
dataset = make_dataset(album, 70)

In [10]:
trainX = np.array(dataset[0])
trainY = np.array(dataset[1])

# Now shuffle the training data and swapaxes
idx = np.random.choice(len(trainX), len(trainX))
trainX = trainX[idx]
trainY = trainY[idx]

trainX = preprocess_input(np.float64(trainX)).swapaxes(1, 3).swapaxes(2, 3)
trainY = np_utils.to_categorical(trainY)

# Only swapaxes for testing data
testX = np.array(dataset[2])
testY = np.array(dataset[3])

testX = preprocess_input(np.float64(testX)).swapaxes(1, 3).swapaxes(2, 3)
testY = np_utils.to_categorical(testY)

In [10]:
# plt.imshow(trainX[10][0, :, :])
# plt.show()

### Fit the model

In [1]:
model = getModel(11)

NameError: name 'getModel' is not defined

In [ ]:
history = model.fit(trainX, trainY, batch_size = 16, nb_epoch = 30, validation_data = (testX, testY), shuffle=True)

Train on 1045 samples, validate on 55 samples
Epoch 1/30
